# Telco Data Churn - Analysis (EDA)
-- **Running this notebook is required in order to BOTH download the raw data and create a pruned + cleaned version of it for use on the NN.** --
## Specific Aim of analysis: Prune and Clean Data
Though possible to give a Neural Network a set of raw data and tell it to simply "predict customer churning," analysing the data to specifically look for things that have any kind of correlation with churning and then using those as the input can increase learning speed and accuracy.
Furthermore, understanding the data helps immensely with understanding the results derived by the NN and how the hidden layers of the NN may work.

### Personally, I also just need to learn how to perform EDA and expand my knowledge of technologies.
I believe in the workplace the NN's role is to automate churn prediction after data scientists and analysists have performed EDA to find out what the predictors actually are.

-- ***As detailed in README.md, this notebook allows for downloading of the dataset onto local devices, provided the user's personal 'kaggle.json' has been provided. If it has not, please refer to the readme.*** --


In [96]:
print('jupyter lab test, hello!')

jupyter lab test, hello!


## Downloading the Dataset
The below cell is for downloading the dataset as detailed in the README.

### Troubleshooting - For if the below cell is not running properly
Please restart the notebook and try again. If problems persist, it could be for one of the following reasons:
1. 'kaggle.json' is in the wrong folder: If you do not have a file 'kaggle.json', refer to the file README.md in the root, otherwise make sure that 'kaggle.json' has been correctly placed in the folder '.kaggle' in the root.
2. The code has shifted out of order: Ensure that the line ``os.environ["KAGGLE_CONFIG_DIR"]`` is before KaggleApi is imported. This sets the working directory
3. The code is pointing to the wrong directory: Please use the commented out print statement labelled 'For Troubleshooting' to check where the api is pointing. If it does not match the .kaggle folder detailed here, you may either amend the workingdir so it points to the right place or follow the error and place 'kaggle.json' in the default scanning area


In [97]:
import os
workingdir = os.path.dirname(os.getcwd()) 
print("kaggle.json should be in: "+workingdir+"\\.kaggle")
os.environ["KAGGLE_CONFIG_DIR"] = os.path.join(workingdir,'.kaggle')

from kaggle.api.kaggle_api_extended import KaggleApi

# point the Kaggle CLI to the Kaggle folder, allowing for use of the dataset outside of the Kaggle Website
api = KaggleApi()

# For troubleshooting: print("The variable \"working dir\" should match: "+api.config_dir)
api.authenticate()

api.dataset_download_files(
    'blastchar/telco-customer-churn',
    path='../data/raw',
    unzip=True  # Extract into data/raw/
)
print("Downloaded and unzipped to data/raw/")


kaggle.json should be in: c:\Users\osiam\OneDrive\Documents\GitHub\A-Newbie-Made-Neural-Network\.kaggle
Dataset URL: https://www.kaggle.com/datasets/blastchar/telco-customer-churn
Downloaded and unzipped to data/raw/


In [98]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

In [99]:
df = pd.read_csv('../data/raw/WA_Fn-UseC_-Telco-Customer-Churn.csv')

### Step 1: Data understanding
In order to prune the data for our purposes, we first need to understand what data is included in the CSV

#### Data Overview

In [100]:
df.shape

(7043, 21)

Clearly not as large as a dataset like MNIST, but surely enough for some sort of learning to be done.

In [101]:
#It's generally better to see more columns than less, considering I have no clue what I'm looking for
#There are 21 columns, as found out above
pd.set_option('display.max_columns', 21)
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [103]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [104]:
df.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


Interestingly, TotalCharges is not in the above calculations despite clearly being a numerical field. This will be looked at later.

Also interestingly, SeniorCitizen is a boolean field that was created using binary representation (dtype int64) and therefore included in this numeric calculation, so we can see the average number of senior citizens (0.16% of the population)and its standard deviation.
This may be used for other boolean fields if possible.

In [105]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

At first glance, the columns Gender, SeniorCitizen, Partner, Dependents, Tenure, Contract, MonthlyCharges and TotalCharges seem the most interesting toward our goal of finding churn predictors. 

Numerical columns that can be used for graphs are Tenure, MonthlyCharges and TotalCharges - however TotalCharges has dtype object which can't be used for numerical operations. This is the first thing that will be addressed in the next section.
# Step 2: Data Preparation
Springboarding off of the problem identified in the last section, I aim to make TotalCharges available for numeric operations with matplotlib and seaborn - but as a side benefit, avoid cases of texual numeric nulls in TotalCharges. 

This will be done as follows:
1. Search for and inspect non-numeric entries in the column
2. If non-numerics signify null entries, attempt a safe conversion of the dtype to numeric (coercing invalid strings to NaN), otherwise continue

In [106]:
#The pandas regular expression of the year seen below:
#Whole thing creates a column locating total charges that are not of type float64 and sets a "True" value if such is the case. If the value is normal then it reports "False"
non_numerics = ~df['TotalCharges'].astype(str).str.match(r'^\s*\d+(\.\d+)?\s*$')

#Breaking it down again for my own understanding and satisfaction:
# r'[placeholder text]' initiates a regex, similar to what is used in javaCC - Formal Languages is such a fun class guys please take it
# \s* allows for whitespace at the front
# \d+ means 1 or more denary numbers must follow
# (\.\d+)? means that optionally, a decimal point can follow before a bunch more denary numbers
# $ Signifies the end of the expression


df[non_numerics]['TotalCharges'].value_counts()            #If there are any non numerical entries, search for them for inspection

TotalCharges
    11
Name: count, dtype: int64

In [107]:
# We now inspect the rows with non numerical TotalCharges:
df[non_numerics]

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
488,4472-LVYGI,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,,No
753,3115-CZMZD,Male,0,No,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,,No
936,5709-LVOEQ,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,Yes,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,,No
1082,4367-NUYAO,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,,No
1340,1371-DWPAZ,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,,No
3331,7644-OMVMY,Male,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,,No
3826,3213-VVOLG,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,,No
4380,2520-SGTTA,Female,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,,No
5218,2923-ARZLG,Male,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,,No
6670,4075-WKNIU,Female,0,Yes,Yes,0,Yes,Yes,DSL,No,Yes,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,,No


We see that in all of these rows, the TotalCharges column is just empty. Interesting patterns are None of them are senior citizens; All of them are dependents with a 0 year tenure; All but 1 are on a 2 year contract and None of them are recorded as having Churned.

Though the significance of this is to be seen, we first deal with the column datatype conversion:

In [108]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
# Sanity Check
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [109]:
# Then finally, repeating the earlier operation to check again
df.isnull().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

Perfect.

The the nulls may have a correlation with churn in the end, the low number in comparison to the entire dataset (11/7043 ~ 0.0016% of values) implies they are moreso just outliers. Their existence will be held in mind for future analysis, though not immediately acted upon.

In [110]:
df.describe()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
count,7043.000000,7043.000000,7043.000000,7032.000000
mean,0.162147,32.371149,64.761692,2283.300441
std,0.368612,24.559481,30.090047,2266.771362
min,0.000000,0.000000,18.250000,18.800000
25%,0.000000,9.000000,35.500000,401.450000
50%,0.000000,29.000000,70.350000,1397.475000
75%,0.000000,55.000000,89.850000,3794.737500
max,1.000000,72.000000,118.750000,8684.800000


In [117]:
#Check if there are duplicates
df.loc[df.duplicated()]

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn


In [ ]:
#Check specifically customerIDs, since duplicates of other values are more or less expected
df.duplicated(subset=['customerID']).sum()

0

# Step 3: Feature Understanding/Univariate Analysis
